# Analisis Tuntutan Perbelanjaan

Notebook ini menunjukkan cara untuk mencipta ejen yang menggunakan plugin untuk memproses perbelanjaan perjalanan daripada imej resit tempatan, menghasilkan emel tuntutan perbelanjaan, dan memvisualisasikan data perbelanjaan menggunakan carta pai. Ejen memilih fungsi secara dinamik berdasarkan konteks tugas.

Langkah-langkah:
1. OCR Agent memproses imej resit tempatan dan mengekstrak data perbelanjaan perjalanan.
2. Email Agent menghasilkan emel tuntutan perbelanjaan.

### Contoh senario perbelanjaan perjalanan:
Bayangkan anda seorang pekerja yang melakukan perjalanan untuk menghadiri mesyuarat perniagaan di bandar lain. Syarikat anda mempunyai polisi untuk membayar balik semua perbelanjaan perjalanan yang munasabah. Berikut adalah pecahan potensi perbelanjaan perjalanan:
- Pengangkutan:
Tambang penerbangan untuk perjalanan pergi balik dari bandar asal anda ke bandar destinasi.
Perkhidmatan teksi atau e-hailing ke dan dari lapangan terbang.
Pengangkutan tempatan di bandar destinasi (seperti pengangkutan awam, kereta sewa, atau teksi).

- Penginapan:
Penginapan hotel selama tiga malam di hotel perniagaan sederhana yang berdekatan dengan lokasi mesyuarat.

- Makanan:
Elaun makanan harian untuk sarapan, makan tengah hari, dan makan malam, berdasarkan polisi per diem syarikat.

- Perbelanjaan Lain-lain:
Bayaran parkir di lapangan terbang.
Caj akses internet di hotel.
Tip atau caj perkhidmatan kecil.

- Dokumentasi:
Anda menyerahkan semua resit (penerbangan, teksi, hotel, makanan, dll.) dan laporan perbelanjaan yang lengkap untuk pembayaran balik.


## Import perpustakaan yang diperlukan

Import perpustakaan dan modul yang diperlukan untuk buku nota.


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## Definisi Model Perbelanjaan

Cipta model Pydantic untuk perbelanjaan individu dan kelas ExpenseFormatter untuk menukar pertanyaan pengguna kepada data perbelanjaan yang berstruktur.

Setiap perbelanjaan akan diwakili dalam format:
`{'date': '07-Mar-2025', 'description': 'penerbangan ke destinasi', 'amount': 675.99, 'category': 'Pengangkutan'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## Mendefinisikan Ejen - Menjana E-mel

Cipta kelas ejen untuk menjana e-mel bagi menghantar tuntutan perbelanjaan.
- Ejen ini menggunakan penghias `kernel_function` untuk mendefinisikan fungsi yang menjana e-mel bagi menghantar tuntutan perbelanjaan.
- Ia mengira jumlah keseluruhan perbelanjaan dan memformat butiran tersebut ke dalam badan e-mel.


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# Ejen untuk Mengekstrak Perbelanjaan Perjalanan dari Imej Resit

Buat kelas ejen untuk mengekstrak perbelanjaan perjalanan daripada imej resit.
- Ejen ini menggunakan dekorator `kernel_function` untuk mendefinisikan fungsi yang mengekstrak perbelanjaan perjalanan daripada imej resit.
- Tukarkan imej resit kepada teks menggunakan OCR (Pengecaman Aksara Optik) dan ekstrak maklumat yang berkaitan seperti tarikh, penerangan, jumlah, dan kategori.


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## Memproses Perbelanjaan

Tentukan fungsi asinkron untuk memproses perbelanjaan dengan mencipta dan mendaftarkan ejen yang diperlukan, kemudian mengaktifkannya.
- Fungsi ini memproses perbelanjaan dengan memuatkan pembolehubah persekitaran, mencipta ejen yang diperlukan, dan mendaftarkan mereka sebagai plugin.
- Ia mencipta sembang kumpulan dengan dua ejen dan menghantar mesej arahan untuk menghasilkan emel dan carta pai berdasarkan data perbelanjaan.
- Ia menguruskan sebarang ralat yang berlaku semasa pengaktifan sembang dan memastikan pembersihan ejen dilakukan dengan betul.


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## Fungsi utama

Tentukan fungsi utama untuk membersihkan konsol dan menjalankan fungsi `process_expenses` secara asinkron.


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
